# 1. 利用EULUC数据裁剪居民小区
- 提取building中是居民区的区域
- 提取小区aoi中是居民区的区域

# 2. 给每个building要素计算人口等信息
- 给每个building要素连接街道字段
- 给每个building要素连接小区字段
- 计算每个building的几何面积
- 计算每个buidling的总面积 面积*楼层数 (总面积是三维叠加的面积)
- 计算每个building的总面积占全街道总面积(buidling group by)的比重
- 计算每个building的人口数量, 总面积比重*全街道总人口
    - 相同方法计算,外来人口,男性,女性,三种年龄段,少数民族等

In [1]:
# 初始化依赖包与qgis
import os
import numpy as np
import pandas as pd


import geopandas as gpd
import matplotlib.pyplot as plt

# import pyqgis
from qgis.core import *

# Supply path to qgis install location
QgsApplication.setPrefixPath('/Applications/QGIS-LTR.app/Contents/MacOS', True)

# Create a reference to the QgsApplication.  Setting the
# second argument to False disables the GUI.
qgs = QgsApplication([], False)

# Load providers
qgs.initQgis()

# Write your code here to load some layers, use processing
# algorithms, etc.

# Finally, exitQgis() is called to remove the
# provider and layer registries from memory


from qgis.analysis import QgsNativeAlgorithms
import processing
from processing.core.Processing import Processing
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
qgs.exitQgis()


# 获取属性表的函数
def getAttributeTable(vlayer):
    """ get attribute table of a vector layer.
    
    Args:
        vlayer : (QgsVectorLayer instance). vector layer

    returns:
         pandas DataFrame: attribute table
    """
    attribute_dict={}
    
    for field in vlayer.fields(): # 初始化各要素属性为空列表
        attribute_dict[field.name()] = []
        
    for feature in vlayer.getFeatures(): # 遍历矢量图层中个要素
        for field in vlayer.fields(): # 遍历各属性字段
            attribute_dict[field.name()].append(feature[field.name()]) # 添加属性字段值进入字典
            
            
    return pd.DataFrame(attribute_dict) #返回DataFrame

Logged warning: Duplicate provider native registered


In [71]:
input_path = '/Users/oo/Desktop/5.Learning/esri-ces-contest/Codes/temp'
pd.set_option('display.max_columns', None)
getAttributeTable(QgsVectorLayer(r'{}/sh_main_building_res_merge.shp'.format(input_path))).info()
getAttributeTable(QgsVectorLayer(r'{}/sh_main_building_res_merge.shp'.format(input_path))).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196855 entries, 0 to 196854
Data columns (total 26 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FLOOR       196855 non-null  object 
 1   area        196855 non-null  float64
 2   id          196855 non-null  int64  
 3   FLOOR_2     196855 non-null  object 
 4   area_2      196855 non-null  object 
 5   id_2        196855 non-null  object 
 6   townName    196855 non-null  object 
 7   towncode    196855 non-null  object 
 8   aoiname     196855 non-null  object 
 9   aoijnhppri  196855 non-null  object 
 10  total_area  196855 non-null  object 
 11  town_tot_a  196855 non-null  object 
 12  area_prop   196855 non-null  object 
 13  tot_pop     196855 non-null  object 
 14  urban_rat   196855 non-null  object 
 15  birth_rate  196855 non-null  object 
 16  death_rate  196855 non-null  object 
 17  pop_dens    196855 non-null  object 
 18  build_pop   196855 non-null  object 
 19  bu

,FLOOR,area,id,FLOOR_2,area_2,id_2,townName,towncode,aoiname,aoijnhppri,total_area,town_tot_a,area_prop,tot_pop,urban_rat,birth_rate,death_rate,pop_dens,build_pop,build_for_,build_male,build_fema,build_unde,build_betw,build_beyo,build_mino
0,8,14.534,0,8,14.534,0,华漕镇,310112000000.0,御锦商务大厦,36154,116.272,6521272.558,0.000018,193800,0.797,1,1,6881,3.455385954,2.537159036,1.847151625,1.60823433,0.316690224,2.982900259,0.15538539,0.034339904
1,8,332.205,1,8,332.205,1,华漕镇,310112000000.0,御锦商务大厦,36154,2657.64,6521272.558,0.000408,193800,0.797,1,1,6881,78.98008056,57.99208186,42.22051778,36.75956278,7.238618116,68.18043074,3.551658422,0.784910398
2,4,814.510,2,4,814.51,2,华漕镇,310112000000.0,NULL,NULL,3258.04,6521272.558,0.0005,193800,0.797,1,1,6881,96.82284345,71.09334687,51.75875429,45.06408916,8.873928511,83.58339376,4.354030344,0.962233212
3,8,293.789,3,8,293.789,3,华漕镇,310112000000.0,御锦商务大厦,36154,2350.312,6521272.558,0.00036,193800,0.797,1,1,6881,69.84686831,51.28591002,37.33816077,32.50870754,6.401548375,60.29608395,3.140946632,0.6941438
4,4,569.908,4,4,569.908,4,华漕镇,310112000000.0,NULL,NULL,2279.632,6521272.558,0.00035,193800,0.797,1,1,6881,67.74639116,49.7436092,36.21530508,31.53108608,6.209037151,58.48282374,3.046490191,0.673269089


In [18]:


input_path = '/Users/oo/Desktop/5.Learning/esri-ces-contest/Codes/input_data/sh_main'
sh_main_town_filepath = r'{}/sh_main_town.shp'.format(input_path)
sh_main_building_res_filepath = r'{}/sh_main_building_res.shp'.format(input_path)
sh_main_aoi_price_res_filepath = r'{}/sh_main_aoi_price_res.shp'.format(input_path)

sh_pop_town_table_filepath = r'/Users/oo/Desktop/5.Learning/esri-ces-contest/Codes/input_data/SH_Pop_Town.csv'


sh_main_town = QgsVectorLayer(sh_main_town_filepath, "Shanghai main town", "ogr")
sh_main_building_res = QgsVectorLayer(sh_main_building_res_filepath, "Shanghai building res", "ogr")
sh_main_aoi_price_res = QgsVectorLayer(sh_main_aoi_price_res_filepath, "Shanghai main aoi price", "ogr")

sh_pop_town_table = pd.read_csv(sh_pop_town_table_filepath, encoding='utf-8')
sh_main_town_attr = getAttributeTable(sh_main_town)
sh_main_building_res_attr = getAttributeTable(sh_main_building_res)
sh_main_aoi_price_res_attr = getAttributeTable(sh_main_aoi_price_res)

## 将building属性与街道字段空间连接

In [ ]:
temp_path = '/Users/oo/Desktop/5.Learning/esri-ces-contest/Codes/temp'
building_town_join = {'INPUT':sh_main_building_res, 
                      'JOIN':sh_main_town, 
                      'PREDICATE':0, 
                      'METHOD':0, 
                      'PREFIX':'town',
                      'OUTPUT': '{}/building_town_join.shp'.format(temp_path)}

#processing.run('native:joinattributesbylocation', building_town_join)

In [28]:
build_town = QgsVectorLayer('{}/building_town_join.shp'.format(temp_path, "Building Town Join", "ogr"))
getAttributeTable(build_town).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200718 entries, 0 to 200717
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FLOOR       200718 non-null  object 
 1   area        200718 non-null  float64
 2   id          200718 non-null  int64  
 3   towngml_id  200718 non-null  object 
 4   townName    200718 non-null  object 
 5   townlayer   200718 non-null  object 
 6   towncode    200718 non-null  object 
 7   towngrade   200718 non-null  object 
 8   townx       200718 non-null  object 
 9   towny       200718 non-null  object 
 10  town面积      200718 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 16.8+ MB


In [31]:
getAttributeTable(build_town).head()

,FLOOR,area,id,towngml_id,townName,townlayer,towncode,towngrade,townx,towny,town面积
0,8,767.290,4739,layer_township_pg.1,江桥镇,乡镇,310114118000,4,-14877.6,916.346,12408100.0
1,8,359.259,4740,layer_township_pg.1,江桥镇,乡镇,310114118000,4,-14877.6,916.346,12408100.0
2,8,307.911,4741,layer_township_pg.1,江桥镇,乡镇,310114118000,4,-14877.6,916.346,12408100.0
3,8,435.484,4742,layer_township_pg.1,江桥镇,乡镇,310114118000,4,-14877.6,916.346,12408100.0
4,8,331.800,4743,layer_township_pg.1,江桥镇,乡镇,310114118000,4,-14877.6,916.346,12408100.0


## 将building要素与小区面aoi连接

In [32]:
building_town_aoi_join = {'INPUT':build_town, 
                      'JOIN':sh_main_aoi_price_res, 
                      'PREDICATE':0, 
                      'PREFIX':'aoi',
                      'OUTPUT': '{}/building_town_aoi_join.shp'.format(temp_path)}

#processing.run('native:joinattributesbylocation', building_town_aoi_join)
build_town_aoi = QgsVectorLayer('{}/building_town_aoi_join.shp'.format(temp_path, "Building Town aoi Join", "ogr"))
build_town_aoi_attr = getAttributeTable(build_town_aoi)

build_town_aoi_attr = build_town_aoi_attr[['FLOOR', 'area', 'id', 'townName', 'towncode', 'aoiname', 'aoijnhppri']]

## 计算层数与总面积

In [ ]:
build_town_aoi_attr['FLOOR'] = build_town_aoi_attr['FLOOR'].apply(lambda row: int(row))
build_town_aoi_attr['total_area'] = build_town_aoi_attr['FLOOR'] * build_town_aoi_attr['area']
build_town_aoi_attr['town_tot_area'] = build_town_aoi_attr.groupby('towncode')['total_area'].transform(sum)
build_town_aoi_attr['area_prop'] = build_town_aoi_attr['total_area'] / build_town_aoi_attr['town_tot_area']
build_town_aoi_attr.head()

In [50]:
sh_pop_town_table = sh_pop_town_table[['Name_town', 'tot_pop', 'urban_rat', 'for_res', 'male', 'female', 'under_14', 
                   'between_15_64', 'beyond_65', 'minor', 'birth_rate', 'death_rate', 'pop_dens']]

sh_pop_town_table = sh_pop_town_table.fillna(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name_town      251 non-null    object 
 1   tot_pop        122 non-null    float64
 2   urban_rat      117 non-null    float64
 3   for_res        114 non-null    float64
 4   male           122 non-null    float64
 5   female         122 non-null    float64
 6   under_14       121 non-null    float64
 7   between_15_64  121 non-null    float64
 8   beyond_65      121 non-null    float64
 9   minor          120 non-null    float64
 10  birth_rate     120 non-null    float64
 11  death_rate     120 non-null    float64
 12  pop_dens       121 non-null    float64
dtypes: float64(12), object(1)
memory usage: 25.6+ KB


## 合并buiding要素与之前整理的街道人口数据

In [53]:
build_town_aoi_attr_merge = build_town_aoi_attr.merge(sh_pop_town_table, left_on='townName', right_on='Name_town', how='left')
build_town_aoi_attr_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212167 entries, 0 to 212166
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FLOOR          212167 non-null  int64  
 1   area           212167 non-null  float64
 2   id             212167 non-null  int64  
 3   townName       212167 non-null  object 
 4   towncode       212167 non-null  object 
 5   aoiname        212167 non-null  object 
 6   aoijnhppri     212167 non-null  object 
 7   total_area     212167 non-null  float64
 8   town_tot_area  212167 non-null  float64
 9   area_prop      212167 non-null  float64
 10  Name_town      212087 non-null  object 
 11  tot_pop        212087 non-null  float64
 12  urban_rat      212087 non-null  float64
 13  for_res        212087 non-null  float64
 14  male           212087 non-null  float64
 15  female         212087 non-null  float64
 16  under_14       212087 non-null  float64
 17  between_15_64  212087 non-nul

In [ ]:
# 没匹配上的全部干掉
build_town_aoi_attr_merge[build_town_aoi_attr_merge['urban_rat'].isnull()]

In [57]:
build_town_aoi_attr_merge = build_town_aoi_attr_merge.dropna()
build_town_aoi_attr_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212087 entries, 0 to 212157
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FLOOR          212087 non-null  int64  
 1   area           212087 non-null  float64
 2   id             212087 non-null  int64  
 3   townName       212087 non-null  object 
 4   towncode       212087 non-null  object 
 5   aoiname        212087 non-null  object 
 6   aoijnhppri     212087 non-null  object 
 7   total_area     212087 non-null  float64
 8   town_tot_area  212087 non-null  float64
 9   area_prop      212087 non-null  float64
 10  Name_town      212087 non-null  object 
 11  tot_pop        212087 non-null  float64
 12  urban_rat      212087 non-null  float64
 13  for_res        212087 non-null  float64
 14  male           212087 non-null  float64
 15  female         212087 non-null  float64
 16  under_14       212087 non-null  float64
 17  between_15_64  212087 non-nul

In [58]:
# id重复的全部干掉
build_town_aoi_attr_merge = build_town_aoi_attr_merge.drop_duplicates(subset=['id'])
build_town_aoi_attr_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196782 entries, 0 to 212157
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FLOOR          196782 non-null  int64  
 1   area           196782 non-null  float64
 2   id             196782 non-null  int64  
 3   townName       196782 non-null  object 
 4   towncode       196782 non-null  object 
 5   aoiname        196782 non-null  object 
 6   aoijnhppri     196782 non-null  object 
 7   total_area     196782 non-null  float64
 8   town_tot_area  196782 non-null  float64
 9   area_prop      196782 non-null  float64
 10  Name_town      196782 non-null  object 
 11  tot_pop        196782 non-null  float64
 12  urban_rat      196782 non-null  float64
 13  for_res        196782 non-null  float64
 14  male           196782 non-null  float64
 15  female         196782 non-null  float64
 16  under_14       196782 non-null  float64
 17  between_15_64  196782 non-nul

## 计算相关字段

In [65]:
build_town_aoi_attr_merge['build_pop'] = build_town_aoi_attr_merge['tot_pop'] * build_town_aoi_attr_merge['area_prop']
build_town_aoi_attr_merge['build_for_res'] = build_town_aoi_attr_merge['for_res'] * build_town_aoi_attr_merge['area_prop']
build_town_aoi_attr_merge['build_male'] = build_town_aoi_attr_merge['male'] * build_town_aoi_attr_merge['area_prop']
build_town_aoi_attr_merge['build_female'] = build_town_aoi_attr_merge['female'] * build_town_aoi_attr_merge['area_prop']
build_town_aoi_attr_merge['build_under_14'] = build_town_aoi_attr_merge['under_14'] * build_town_aoi_attr_merge['area_prop']
build_town_aoi_attr_merge['build_between_15_64'] = build_town_aoi_attr_merge['between_15_64'] * build_town_aoi_attr_merge['area_prop']
build_town_aoi_attr_merge['build_beyond_65'] = build_town_aoi_attr_merge['beyond_65'] * build_town_aoi_attr_merge['area_prop']
build_town_aoi_attr_merge['build_minor'] = build_town_aoi_attr_merge['minor'] * build_town_aoi_attr_merge['area_prop']

In [66]:
build_town_aoi_attr_merge.head()

,FLOOR,area,id,townName,towncode,aoiname,aoijnhppri,total_area,town_tot_area,area_prop,Name_town,tot_pop,urban_rat,for_res,male,female,under_14,between_15_64,beyond_65,minor,birth_rate,death_rate,pop_dens,build_pop,build_for_res,build_male,build_female,build_under_14,build_between_15_64,build_beyond_65,build_minor
0,8,145.777,128275,高境镇,310113111000,上海财经大学-研究生公寓,56987,1166.216,1.802280e+07,0.000065,高境镇,127500.0,1.0,32037.0,65200.0,62300.0,10040.0,104800.0,12703.0,1093.0,0.00387,0.00507,17959.0,8.250246,2.073044,4.218950,4.031297,0.649666,6.781379,0.821983,0.070726
1,24,349.867,128300,高境镇,310113111000,上海财经大学-研究生公寓,56987,8396.808,1.802280e+07,0.000466,高境镇,127500.0,1.0,32037.0,65200.0,62300.0,10040.0,104800.0,12703.0,1093.0,0.00387,0.00507,17959.0,59.402147,14.926013,30.376628,29.025520,4.677628,48.826236,5.918317,0.509228
2,24,351.423,128301,高境镇,310113111000,上海财经大学-研究生公寓,56987,8434.152,1.802280e+07,0.000468,高境镇,127500.0,1.0,32037.0,65200.0,62300.0,10040.0,104800.0,12703.0,1093.0,0.00387,0.00507,17959.0,59.666333,14.992395,30.511725,29.154608,4.698431,49.043386,5.944639,0.511493
3,24,349.859,128302,高境镇,310113111000,上海财经大学-研究生公寓,56987,8396.616,1.802280e+07,0.000466,高境镇,127500.0,1.0,32037.0,65200.0,62300.0,10040.0,104800.0,12703.0,1093.0,0.00387,0.00507,17959.0,59.400789,14.925671,30.375933,29.024856,4.677521,48.825119,5.918182,0.509216
4,4,94.754,130405,高境镇,310113111000,上海财经大学-研究生公寓,56987,379.016,1.802280e+07,0.000021,高境镇,127500.0,1.0,32037.0,65200.0,62300.0,10040.0,104800.0,12703.0,1093.0,0.00387,0.00507,17959.0,2.681300,0.673732,1.371143,1.310157,0.211139,2.203924,0.267142,0.022986


In [67]:
#build_town_aoi_attr_merge.to_csv('build_town_aoi_attr_merge.csv', encoding='utf_8_sig')

In [68]:
build_town_aoi_attr_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196782 entries, 0 to 212157
Data columns (total 31 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   FLOOR                196782 non-null  int64  
 1   area                 196782 non-null  float64
 2   id                   196782 non-null  int64  
 3   townName             196782 non-null  object 
 4   towncode             196782 non-null  object 
 5   aoiname              196782 non-null  object 
 6   aoijnhppri           196782 non-null  object 
 7   total_area           196782 non-null  float64
 8   town_tot_area        196782 non-null  float64
 9   area_prop            196782 non-null  float64
 10  Name_town            196782 non-null  object 
 11  tot_pop              196782 non-null  float64
 12  urban_rat            196782 non-null  float64
 13  for_res              196782 non-null  float64
 14  male                 196782 non-null  float64
 15  female           

In [14]:
def decorate(func):
    a = 'a'
    b = 'b'
    print(a)
    def closure(*args):
        c = 'c'
        print(a)
        print(c)
        
        return func(*args)
    return closure

@decorate
def my_func(d, e):
    print(d)
    print(e)



d = 'd'
e = 'e'
my_func(d, e)

a
a
c
d
e


In [22]:
def decorate_xxoo(who):
    
    def func1(function):
        
        
        def closure_func(*args):
            if who == 'oo':
                print('oo Love xx!')
                
            if who == 'xx':
                print('xx 打 oo')
        
        
            return function(*args)
        
        return closure_func
    
    
    
    
    
    
    return func1





@decorate_xxoo('oo')
def oo_love_xx(a):
    print('oo',a)
    
@decorate_xxoo('xx')
def xx_love_oo(a):
    print('xx',a)
    
    
a = 10086
oo_love_xx(a)

xx_love_oo(a)


oo Love xx!
oo 10086
xx 打 oo
xx 10086
